# Uses https://github.com/lcylmhlcy/CartoonGAN-Pytorch and its pretrained models (or your own custom models) to generate frame-by-frame baselines.
# Requires installation of ffmpeg.

In [2]:
import csv
import subprocess
import math
import json
import os
import shlex
import cv2 
import shutil
from tqdm import tqdm
import numpy as np

In [20]:
#create the directory if it doesn't exist, empty it if it does
def cleardir(dirpath):
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)
    try:
        os.mkdir(dirpath)
    except OSError:
        print ("Creation of the directory %s failed" % dirpath)
        print(OSError)
    else:
        print ("Successfully created the directory %s " % dirpath)

In [21]:
# https://github.com/c0decracker/video-splitter/blob/master/ffmpeg-split.py

def get_video_length(filename):

    output = subprocess.check_output(("ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", filename)).strip()
    video_length = int(float(output))
    print ("Video length in seconds: "+str(video_length))

    return video_length

def ceildiv(a, b):
    return int(math.ceil(a / float(b)))

def split_by_seconds(filename, split_length, vcodec="copy", acodec="copy",
                     extra="", video_length=None, **kwargs):
    if split_length and split_length <= 0:
        print("Split length can't be 0")
        raise SystemExit

    if not video_length:
        video_length = get_video_length(filename)
    split_count = ceildiv(video_length, split_length)
    if(split_count == 1):
        print("Video length is less then the target split length.")
        raise SystemExit

#     split_cmd = ["ffmpeg", "-i", filename, "-vcodec", vcodec, "-acodec", acodec] + shlex.split(extra)
    split_cmd = ["ffmpeg"] + shlex.split(extra)
    try:
        filebase = ".".join(filename.split(".")[:-1])
        shortname = filebase.split('/')[-1]
        fileext = filename.split(".")[-1]
    except IndexError as e:
        raise IndexError("No . in filename. Error: " + str(e))
    
    path = "vid_short/" + shortname
    cleardir(path)
        
    for n in range(0, split_count):
        split_args = []
        if n == 0:
            split_start = 0
        else:
            split_start = split_length * n

#         split_args += ["-ss", str(split_start), "-t", str(split_length),
#                        filebase + "-" + str(n+1) + "-of-" + \
#                         str(split_count) + "." + fileext]
        split_args += ["-ss", str(split_start), "-t", str(split_length), "-i", filename, 
                       "-vcodec", vcodec, "-acodec", acodec,
                       "vid_short/" + shortname + "/" + shortname + str(n+1) + "." + fileext]
        print ("About to run: "+" ".join(split_cmd+split_args))
        subprocess.check_output(split_cmd+split_args)

In [31]:
# https://www.geeksforgeeks.org/python-program-extract-frames-using-opencv/
# Program To Read video 
# and Extract Frames 
  
# Function to extract frames 
def FrameCapture(inpath, outfolder, div = 24, startFrame = 0): 
      
    cleardir(outfolder)
    
    # Path to video file 
    vidObj = cv2.VideoCapture(inpath) 
  
    # Used as counter variable 
    count = startFrame
  
    # checks whether frames were extracted 
    success = 1
  
    while success: 
        vidObj.set(cv2.CAP_PROP_POS_MSEC,(count*1000/div))
        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read() 
        if not success:
            break
        # Saves the frames with frame-count 
        cv2.imwrite((outfolder + "/" + "frame%d.jpg") % count, image) 
  
        count += 1
  


In [23]:
#rejoin all images in folder according to pattern into video at fps
def rejoin(folder, pattern, outfile, fps):
    cmd = "ffmpeg -r " + str(fps) + " -i " + folder + '/' + pattern + " -vcodec mpeg4 -y -c:v libx264 -pix_fmt yuv420p " + outfile
    
    print(cmd)
    os.system(cmd)
#     os.system("ffmpeg -r 1 -i frame%01d.jpg -vcodec mpeg4 -y movie.mp4")

In [8]:
split_start = 0 #start second
split_length = 26 #end second
outfile = "CynDance" #file identifier
filename = "AnimeGAN/vid_data/full/CynDance.mp4" #infile
shortoutfolder = "baseline/short" #outfolder for clipped video
frameoutfolder = "baseline/frames_in" + "/" + outfile #outfolder for video frames
frameoutfolder2 = "baseline/frames_out" + "/" + outfile #outfolder for converted frames
videooutfolder = "baseline/merge/" + outfile #outfolder for merged conver



split_args = ["ffmpeg", "-y", "-ss", str(split_start), "-t", str(split_length), "-i", filename, 
                       "-vcodec", "copy", "-acodec", "copy",
                       shortoutfolder + "/" + outfile + ".mp4"]
print ("About to run: "+" ".join(split_args))
subprocess.check_output(split_args)

FrameCapture(shortoutfolder + "/" + outfile + ".mp4", frameoutfolder)

split_args = ["python", "CartoonGAN-Pytorch/test.py", "--input_dir", frameoutfolder, "--output_dir",
             frameoutfolder2, "--gpu", "0", "--model_path", "CartoonGAN-Pytorch/pretrained_model"]
cleardir(frameoutfolder2)
print ("About to run: "+" ".join(split_args))
subprocess.check_output(split_args)

videooutfolder = "baseline/merge/" + outfile
cleardir(videooutfolder)
rejoin(frameoutfolder2, "frame%01d_Hayao.jpg", videooutfolder + "/" + "Hayao.mp4" ,24)
rejoin(frameoutfolder2, "frame%01d_Hosoda.jpg", videooutfolder + "/" + "Hosoda.mp4" ,24)
rejoin(frameoutfolder2, "frame%01d_Shinkai.jpg", videooutfolder + "/" + "Shinkai.mp4" ,24)
rejoin(frameoutfolder2, "frame%01d_Paprika.jpg", videooutfolder + "/" + "Paprika.mp4" ,24)
rejoin(frameoutfolder2, "frame%01d_Shinkai.jpg", videooutfolder + "/" + "ShinkaiSlow.mp4" ,2)

About to run: ffmpeg -y -ss 0 -t 26 -i AnimeGAN/vid_data/full/CynDance.mp4 -vcodec copy -acodec copy baseline/short/CynDance.mp4
Successfully created the directory baseline/frames_in/CynDance 
Successfully created the directory baseline/frames_out/CynDance 
About to run: python CartoonGAN-Pytorch/test.py --input_dir baseline/frames_in/CynDance --output_dir baseline/frames_out/CynDance --gpu 0 --model_path CartoonGAN-Pytorch/pretrained_model


b'GPU mode\nFinished!\n'